In [22]:
import pandas as pd
import numpy as np
import json 
import re
from geojson import Feature, FeatureCollection, MultiPoint, Point
import uuid
import math

In [47]:
class Vehicle:
    def __init__(
        self,
        id,
        department,
        make,
        model,
        year,
        vehicle_class
        ):
        self.id = id
        self.department = department
        self.make = make
        self.model = model
        self.year = year
        self.vehicle_class = vehicle_class
        self.trips = []

In [4]:
class Trip:
    def __init__(
        self,
        vehicle,
        start_time,
        end_time,
        date,
        ):
        self.id = str(uuid.uuid4())
        self.vehicle = vehicle
        self.start_time = start_time
        self.end_time = end_time
        self.date = date
        self.stops = []
        self.num_stops = 0
        self.distance = 0
        self.total_duration = 0
        self.idle_duration = 0
        self.driving_duration = 0
        self.stopped_duration = 0
        self.geojson = []
        self.show = True

    def add_stop(self, stop):
        #self.stops.append(stop.__dict__)
        stop_properties = {
            'vehicle': self.vehicle,
            'stopDuration': stop.stop_duration,
            'idleDuration': stop.idle_duration,
            'startTime': stop.start_time,
            'endTime': stop.end_time,
            'drivingDuration': stop.driving_duration,
            'distance': stop.distance,
            'stopNum': self.num_stops
        }
        point = Point(stop.location)
        feature = Feature(geometry=point, properties=stop_properties)
        self.geojson.append(feature)
        
        self.num_stops += 1
        self.distance += stop.distance
        self.total_duration += (stop.stop_duration + stop.driving_duration)
        self.idle_duration += stop.idle_duration
        self.driving_duration += stop.driving_duration
        self.stopped_duration += stop.stop_duration


In [5]:
class Department:
    def __init__ (
        self,
        name,
        vehicles
    ):
        self.name = name
        self.num_vehicles = len(vehicles);
        self.vehicles = vehicles


In [6]:
class Stop:
    def __init__(
        self,
        location_lat,
        location_long,
        stop_duration,
        idle_duration,
        driving_duration,
        start_time,
        end_time,
        distance
        ):
        self.location = (location_long, location_lat)
        self.stop_duration = stop_duration
        self.idle_duration = idle_duration
        self.driving_duration = driving_duration
        self.start_time = start_time
        self.end_time = end_time
        self.distance = distance

In [7]:
fleet_df = pd.read_csv('../src/data/vehicle_details/FleetDetails.csv')

In [8]:
fleet_df = fleet_df.fillna('')

In [9]:
fleet_df['VehicleName'] = [str(name).split(' ')[0] for name in fleet_df['VehicleName']]
fleet_df['VehicleName'] = [re.sub('[\W_]+', '', str(name)) for name in fleet_df['VehicleName']]

In [55]:
fleet = []
for idx, row in fleet_df.iterrows():
    vehicle = Vehicle(row['VehicleName'], row['Department'], row['Make'], row['Model'], row['Year'], row['Class'])
    fleet.append(vehicle)

In [11]:
dept_obj_arr = []
groups = fleet_df.groupby('Department')
for name, rows in groups:
    dept_vehicles = []
    for idx, row in rows.iterrows():
        dept_vehicles.append(row['VehicleName'])
    dept_obj_arr.append(Department(name, dept_vehicles).__dict__)

In [56]:
trips_df = pd.read_csv('sept-oct_trip_latlng.csv')
# trips_df = trips_df.rename(columns={'Unnamed: 0': 'idx'})
# trips_df['trip_num'] = 0

In [2]:
# grouped = trips_df.groupby('Device')
# trip_num = 0
# for name, group in grouped:
#     last_row = group.iloc[0]
#     for i, row in group.iterrows():
#         if row.idx - last_row.idx > 1:
#             trip_num += 1
#         trips_df['trip_num'].iloc[row.idx] = trip_num
#         last_row = row
#     trip_num += 1

In [57]:
trip_objs = []
trips = []
grouped = trips_df.groupby('TripNum')
for name, group in grouped:
    trip = Trip(group.iloc[0]['Device'], group.iloc[0]['StopStart'], group.iloc[-1]['StopEnd'], group.iloc[0]['Date'])
    for i, row in group.iterrows():
        # if math.isnan(row.lat) or math.isnan(row.long):
        #     r_row = trips_df.sample(n=1).iloc[0]
        #     while not math.isnan(r_row.lat) and not math.isnan(r_row.long):
        #         r_row = trips_df.sample(n=1).iloc[0]
        #     lat = r_row.lat
        #     lng = r_row.long
        # else:
        #     lat = row.lat
        #     lng = row.long
        if not math.isnan(row.lat) and not math.isnan(row.long):
            stop = Stop(row['lat'], row['long'], row['StopDuration'], row['IdleDuration'], row['DrivingDuration'], row['StopStart'], row['StopEnd'], row['Distance'])
            trip.add_stop(stop)
    trips.append(trip)
    trip_objs.append(trip.__dict__)



In [58]:
for vehicle in fleet:
    for trip in trips:
        if vehicle.id == trip.vehicle:
            trip_obj = {
                'idle_duration': trip.idle_duration,
                'total_duration': trip.total_duration,
                'driving_duration': trip.driving_duration,
                'stopped_duration': trip.stopped_duration,
                'distance': trip.distance,
                'start_time': trip.start_time,
                'end_time': trip.end_time,
                'num_stops': trip.num_stops,
            }
            vehicle.trips.append(trip_obj)

In [59]:
fleet_obj_arr = []
for vehicle in fleet:
    fleet_obj_arr.append(vehicle.__dict__)


In [43]:
stops_obj_arr = []
for i, row in trips_df.iterrows():
    if not math.isnan(row.lat) and not math.isnan(row.long):
        stop = Stop(row['lat'], row['long'], row['StopDuration'], row['IdleDuration'], row['DrivingDuration'], row['StopStart'], row['StopEnd'], row['Distance'])
        stop.lat = row['lat']
        stop.lng = row['long']
        stops_obj_arr.append(stop.__dict__)

In [45]:
stops_obj = {'stops': stops_obj_arr}

with open("heatmap_data.json", "w") as outfile:
    json.dump(stops_obj, outfile)

In [60]:
fleet_details_obj = {'vehicles': fleet_obj_arr}
trip_details_obj = {'trips': trip_objs}
dept_details_obj = {'departments': dept_obj_arr}

In [34]:
with open("fleet_details.json", "w") as outfile:
    json.dump(fleet_details_obj, outfile)

with open("trips_sept-oct.json", "w") as outfile:
    json.dump(trip_details_obj, outfile)

    
with open("depts_9-27_9-29.json", "w") as outfile:
    json.dump(dept_details_obj, outfile)